# Project 3
## CS 7324
#### Jennifer Carballo & Amory Weinzierl

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#### load in datasets

In [ ]:
# load in real estate dataset

real_estate_df = pd.read_csv("data/WakeCountyHousing.csv")

# display df
real_estate_df.head()

In [ ]:
# load in reddit upvote dataset - test and train

reddit_upvotes_train_df = pd.read_csv("data/train_NIR5Yl1.csv")
reddit_upvotes_test_df = pd.read_csv("data/test_8i3B3FC.csv")

reddit_upvotes_train_df.head()

In [ ]:
reddit_upvotes_test_df.head()

In [ ]:
# load in uber fare dataset

uber_fares_df = pd.read_csv("data/uber.csv")
uber_fares_df = uber_fares_df.rename(columns={'Unnamed: 0': 'index'})
uber_fares_df = uber_fares_df.set_index("index")

uber_fares_df.head()

#### explore data

##### explore real estate data

In [ ]:
real_estate_df.info()

In [ ]:
real_estate_df.describe()

In [ ]:
real_estate_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
real_estate_df.isna().sum().sum()

In [ ]:
for columnName in real_estate_df:
    print(columnName, real_estate_df[columnName].dtypes)

In [ ]:
for columnName in real_estate_df:
    print(columnName, real_estate_df[columnName].isna().sum())

##### explore reddit upvote training data

In [ ]:
reddit_upvotes_train_df.info()

In [ ]:
reddit_upvotes_train_df.describe()

In [ ]:
reddit_upvotes_train_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
reddit_upvotes_train_df.isna().sum().sum()

In [ ]:
for columnName in reddit_upvotes_train_df:
    print(columnName, reddit_upvotes_train_df[columnName].dtypes)

##### explore uber fares data

In [ ]:
uber_fares_df.info()

In [ ]:
uber_fares_df.describe()

In [ ]:
uber_fares_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
uber_fares_df.isna().sum().sum()

In [ ]:
for columnName in uber_fares_df:
    print(columnName, uber_fares_df[columnName].isna().sum())

In [ ]:
# since only two datapoints in entire dataframe are null, simply remove rows where null data is
uber_fares_df = uber_fares_df.dropna()

In [ ]:
uber_fares_df.isna().sum().sum()

In [ ]:
for columnName in uber_fares_df:
    print(columnName, uber_fares_df[columnName].dtypes)

#### executing tasks on real estate df

In [ ]:
# replace missing values

for columnName in real_estate_df:
    print(columnName, real_estate_df[columnName].isna().sum())

In [ ]:
real_estate_df['Bath'].value_counts()

In [ ]:
real_estate_df[real_estate_df['Bath'].isna()]

In [ ]:
real_estate_df[(real_estate_df['Bath'].isna()) & (real_estate_df['Design_Style'] == 'Conventional')]

In [ ]:
real_estate_df[real_estate_df['Design_Style'] == "Conventional"]["Bath"].value_counts()

In [ ]:
real_estate_df[real_estate_df['Design_Style'] == "Conventional"]["Bath"].mode()[0]

In [ ]:
real_estate_df.loc[real_estate_df['Design_Style'] == "Conventional", 'Bath'] = real_estate_df['Bath'].fillna(real_estate_df[real_estate_df['Design_Style'] == "Conventional"]["Bath"].mode()[0])

In [ ]:
real_estate_df[(real_estate_df['Bath'].isna()) & (real_estate_df['Design_Style'] == 'Conventional')]

In [ ]:
real_estate_df[real_estate_df['Design_Style'] == "Condo"]["Bath"].mode()[0]

In [ ]:
real_estate_df.loc[real_estate_df['Design_Style'] == "Condo", 'Bath'] = real_estate_df['Bath'].fillna(real_estate_df[real_estate_df['Design_Style'] == "Condo"]["Bath"].mode()[0])

In [ ]:
real_estate_df[(real_estate_df['Bath'].isna()) & (real_estate_df['Design_Style'] == 'Condo')]

In [ ]:
for columnName in real_estate_df:
    print(columnName, real_estate_df[columnName].isna().sum())

In [ ]:
print(len(real_estate_df))

In [ ]:
real_estate_df = real_estate_df.dropna()

In [ ]:
for columnName in real_estate_df:
    print(columnName, real_estate_df[columnName].isna().sum())

In [ ]:
real_estate_df.isna().sum().sum()

In [ ]:
# utilize ordinal encoder

real_estate_df['Bath'].value_counts()

In [ ]:
real_estate_df['Bath'] = real_estate_df['Bath'].replace('2½ Bath', 4)
real_estate_df['Bath'] = real_estate_df['Bath'].replace('2 Bath', 3)
real_estate_df['Bath'] = real_estate_df['Bath'].replace('3½ Bath', 6)
real_estate_df['Bath'] = real_estate_df['Bath'].replace('3 Bath', 5)
real_estate_df['Bath'] = real_estate_df['Bath'].replace('Other', 0)
real_estate_df['Bath'] = real_estate_df['Bath'].replace('1 Bath', 1)
real_estate_df['Bath'] = real_estate_df['Bath'].replace('1 ½ Bath', 2)

In [ ]:
real_estate_df['Bath'].value_counts()

In [ ]:
# utilize one hot encoder

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown = 'ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(real_estate_df[['Num_Stories']]).toarray())
                          
final_df = real_estate_df.join(encoder_df)

final_df

In [ ]:
real_estate_df.isna().sum().sum()

In [ ]:
# implement custom transformer
# https://medium.com/@benlc77/how-to-write-clean-and-scalable-code-with-custom-transformers-sklearn-pipelines-ecb8e53fe110

from sklearn.base import BaseEstimator, TransformerMixin

class DropFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, variables):
        self.variables = variables
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        X_dropped = X.drop(self.variables, axis = 1)
        self.columns = X_dropped.columns
        return X_dropped
    
transformer = DropFeatureSelector('Month_Year_of_Sale')

In [ ]:
transformer.fit(real_estate_df)

In [ ]:
transformer.transform(real_estate_df)

In [ ]:
# scale/normalize/standardize features using sklearn.preprocessing

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_features = scaler.fit_transform(real_estate_df.Heated_Area.values.reshape(-1,1))
scaled_features

In [ ]:
real_estate_df['Heated_Area'] = scaled_features

In [ ]:
real_estate_df

## Executing tasks on reddit train df

### Preparing Data:

In [ ]:
# checking attribute correlation to upvotes
corr_matrix = reddit_upvotes_train_df.corr()
corr_matrix["Upvotes"].sort_values(ascending=False)

In [ ]:
# checking combo attributes
upvotes_copy = reddit_upvotes_train_df.copy()

upvotes_copy["rep_per_view"] = upvotes_copy["Reputation"] / upvotes_copy["Views"]
upvotes_copy["view_per_rep"] = upvotes_copy["Views"] / upvotes_copy["Reputation"]
upvotes_copy["ans_per_view"] = upvotes_copy["Answers"] / upvotes_copy["Views"]

corr_matrix = upvotes_copy.corr()
corr_matrix["Upvotes"].sort_values(ascending=False)

In [ ]:
# looking at scatter plot for upvotes per views
reddit_upvotes_train_df.plot(kind="scatter", x="Views", y="Upvotes", alpha=0.1)

In [ ]:
# remove categorical attribute tag from df
train_numerical = reddit_upvotes_train_df.copy().drop("Tag", axis=1)
test_numerical = reddit_upvotes_test_df.copy().drop("Tag", axis=1)
train_numerical

### Scaling Data

In [ ]:
from sklearn import preprocessing

scalar = preprocessing.StandardScaler().fit(train_numerical)
X = scalar.transform(train_numerical)
prepared_train = pd.DataFrame(X, columns=train_numerical.columns,
                          index=train_numerical.index)

train_labels = prepared_train["Upvotes"].copy()
prepared_train = prepared_train.copy().drop("Upvotes", axis = 1)
train_labels

In [ ]:
# Train using forest regression
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10,
                             max_depth=10,
                             criterion='squared_error',
                            )
forest_reg.fit(prepared_train, train_labels)

In [ ]:
# trying it on training data
small_data = prepared_train.iloc[:5]
small_labels = train_labels.iloc[:5]
print("Predictions:", forest_reg.predict(small_data))

In [ ]:
print("Labels:", list(small_labels))

### Task #1: K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

x_train = prepared_train[:10000]
y_train = train_labels[:10000]
    
reg_scores = cross_val_score(forest_reg, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
reg_rmse_scores = np.sqrt(-reg_scores)

display_scores(reg_rmse_scores)

### Task #2: StratifiedK-Fold Cross Validation

In [ ]:
# for classifier I will be using the "tag" attribute as the target (also I reduced the data set size to 1000 because my laptop is weak)
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

y_train_cat = reddit_upvotes_train_df["Tag"][:10000]

rnd_clf = RandomForestClassifier()
rnd_clf.fit(x_train, y_train_cat)

cnt = 1
skf = StratifiedKFold(n_splits=3, shuffle=False)
for train_index, test_index in skf.split(x_train, y_train_cat):
    clone_clf = clone(rnd_clf)
    x_train_folds = x_train.iloc[train_index]
    y_train_folds = y_train_cat[train_index]
    
    clone_clf.fit(x_train_folds, y_train_folds)
    y_pred = clone_clf.predict(x_train_folds)
    print("Accuracy for Fold", cnt, ":", accuracy_score(y_train_folds, y_pred))
    cnt += 1

### Task #3: Use sklearn.mean_squared_error and One Other Option to Evaluate Model Performance

#### Mean Squared Error

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = forest_reg.predict(x_train)
reg_mse = mean_squared_error(y_train, predictions)
print("MSE:", reg_mse)
reg_rmse = np.sqrt(reg_mse)
print("RMSE:", reg_rmse)

#### R^2 Score

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_train, predictions)
r2

### Task #4: Generate a Confusion Matrix

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(rnd_clf, x_train, y_train_cat, cv=3)
confusion_matrix(y_train_cat, y_pred)

### Task #5: Generate ROC Curve

In [ ]:
# logistic regression for multi-class classification using a one-vs-rest (must convert into binary classification to do this task)
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = LogisticRegression()
ovr = OneVsRestClassifier(model)
ovr.fit(x_train, y_train_cat)
yhat = ovr.predict(x_train)
yhat

In [ ]:
from sklearn.metrics import roc_curve

y_scores = cross_val_predict(ovr, x_train, yhat, cv=3,
                             method="decision_function")
fpr, tpr, thresholds = roc_curve(yhat, y_scores, pos_label={'a', 'c', 'r', 'j', 'p', 's', 'h', 'o', 'i', 'x'})

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
    
plot_roc_curve(fpr, tpr)
plt.show()

### Task #6: Use Grid Search CV to Tune Hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

new_forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(new_forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(x_train, y_train)

In [ ]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
grid_search.best_estimator_

### Task #7: Use an Ensemble of Methods

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

small_x = x_train[:1000]
small_y = y_train_cat[:1000]

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('sv', svm_clf)],
    voting='hard')
voting_clf.fit(small_x, small_y)

In [ ]:
y_pred = voting_clf.predict(small_x)
print("Ensemble Accuracy: ", accuracy_score(small_y, y_pred))

### Task #8: Evaluate Your System on the Test Data

In [ ]:
final_model = grid_search.best_estimator_

x_test = prepared_train[1000:2000]
y_test = train_labels[1000:2000]

final_model.fit(x_test, y_test)
final_predictions = final_model.predict(x_test)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

### Task #9: Create a Single Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

numeric_features = ['ID', 'Reputation', 'Answers', 'Username', 'Views']
categorical_features = ['Tag']

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
]) 

pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor', RandomForestRegressor(bootstrap=False, max_features=4, n_estimators=10)),
           ])

y_upvotes = reddit_upvotes_train_df["Upvotes"]
x_upvotes = reddit_upvotes_train_df.copy().drop("Upvotes", axis=1)
x_test = x_upvotes[1000:2000]
y_test = y_upvotes[1000:2000]

model = pipeline.fit(x_upvotes, y_upvotes)
final_predictions = model.predict(x_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

#### executing tasks on uber fares df

In [ ]:
# followed along with textbook classification chapter

uber_fares_df

In [ ]:
uber_fares_df.drop(columns=['key'], inplace = True)
uber_fares_df

In [ ]:
uber_fares_df.drop(columns=['pickup_datetime'], inplace = True)
uber_fares_df

In [ ]:
x_data = uber_fares_df[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']]
y_data = uber_fares_df[['fare_amount']]

In [ ]:
# create train and test sets for uber dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.2, shuffle=False)

y_train_greater10 = (y_train > 10)
y_test_greater10 = (y_test > 10)

In [ ]:
# use SGDClassifier
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_greater10)

sgd_y_pred = sgd_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test_greater10, sgd_y_pred))

In [ ]:
# use sklearn.linear_model_LinearRegression
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train_greater10)

print("Score:",lin_reg.score(X_test, y_test_greater10))

In [ ]:
# use sklearn.tree.DecisionTreeRegressor

from sklearn.tree import DecisionTreeRegressor 

regressor = DecisionTreeRegressor(random_state = 0) 

regressor.fit(X_train, y_train_greater10)

print("Score:",regressor.score(X_test, y_test_greater10))

In [ ]:
# use sklearn.ensemble.RandomForestClassifer

from sklearn.ensemble import RandomForestClassifier

rf_clf=RandomForestClassifier(n_estimators=100)

rf_clf.fit(X_train,y_train_greater10)

rf_y_pred=rf_clf.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test_greater10, rf_y_pred))

In [ ]:
# use sklearn.neighbors.KNeighborsClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
import numpy as np

y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_multilabel)

y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)

In [ ]:
print("Accuracy:",accuracy_score(y_multilabel, y_train_knn_pred))

In [ ]:
# use OvO or OvR classifier

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import LogisticRegression

OvO_clf = OneVsOneClassifier(LogisticRegression())
OvO_clf.fit(X_train, y_train_greater10)

y_pred = OvO_clf.predict(X_test)

In [ ]:
print("Accuracy:",accuracy_score(y_test_greater10, y_pred))